In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.datasets import load_files
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

train_path = './train'
validate_path = './val'
test_path = './test'


Using TensorFlow backend.


In [2]:
dim = 299
epochs = 50
learning_rate = 0.0001
batch_size = 16

train_num = 3790
validation_num = 480


In [3]:

weights = os.path.join('', 'weights_resnet.h5')

datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1)


In [4]:
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode='categorical')

validate_generator = datagen.flow_from_directory(
    validate_path,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode='categorical')

Found 3790 images belonging to 12 classes.
Found 480 images belonging to 12 classes.


In [5]:

callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0),
             ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=1),
             ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001,
                               cooldown=0, min_lr=0)]

In [6]:

base_model = ResNet50(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg')
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])


In [7]:

model.fit_generator(train_generator,
                    steps_per_epoch=train_num / batch_size,
                    validation_data=validate_generator,
                    validation_steps=validation_num / batch_size,
                    callbacks=callbacks,
                    epochs=epochs,
                    verbose=1)

Epoch 1/50
237/236 [==============================] - 273s - loss: 0.8925 - acc: 0.7246 - val_loss: 0.6072 - val_acc: 0.8042
Epoch 2/50
237/236 [==============================] - 260s - loss: 0.3434 - acc: 0.8836 - val_loss: 0.3099 - val_acc: 0.8708
Epoch 3/50
237/236 [==============================] - 257s - loss: 0.2668 - acc: 0.9117 - val_loss: 0.3361 - val_acc: 0.8896
Epoch 4/50
237/236 [==============================] - 260s - loss: 0.2137 - acc: 0.9285 - val_loss: 0.2957 - val_acc: 0.9042
Epoch 5/50
237/236 [==============================] - 260s - loss: 0.1889 - acc: 0.9444 - val_loss: 0.2642 - val_acc: 0.9083
Epoch 6/50
237/236 [==============================] - 257s - loss: 0.1599 - acc: 0.9509 - val_loss: 0.3262 - val_acc: 0.9000
Epoch 7/50
237/236 [==============================] - 256s - loss: 0.1656 - acc: 0.9483 - val_loss: 0.6047 - val_acc: 0.8562
Epoch 8/50
237/236 [==============================] - 260s - loss: 0.1377 - acc: 0.9560 - val_loss: 0.2426 - val_acc: 0.9229


In [14]:

test_datagen = ImageDataGenerator()      
generator_test = test_datagen.flow_from_directory(
test_path,
target_size=(dim, dim),
batch_size=batch_size,
class_mode=None,  
shuffle=False)  

Found 480 images belonging to 12 classes.


In [15]:

prediction_test = model.predict_generator(generator_test, verbose=1,steps =30)

30/30 [==============================] - 11s    


In [16]:
print("done")

done


In [17]:
prediction_list = []
for i in range(len(prediction_test)):
    result = np.argmax(prediction_test[i])
    prediction_list.append(result)

count = 0

for i in range(len(prediction_list)):
    if prediction_list[i] == generator_test.classes[i]:
        
        count += 1
print(count/480)

480
480
0.9479166666666666
